# A more Humanly Readable version of Word2Quiz - Written in Jupyter!
I've converted nearly everything from my code to Jupyter for maintenance ease purposes. This will probably not going to get released to the public so I have no idea why I'm writing this right now. lol

In [ ]:
from assets.file_manage import convertDoc2Txt, formatter
from assets.file_manage import filterer, start
from selenium import webdriver

import time

from colorama import Back, Fore, Style
# print(Fore.YELLOW)
import docx2txt
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as exc
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# Load Enviroment Variables
import os
from dotenv import load_dotenv, dotenv_values
load_dotenv()

EMAIL = os.getenv("EMAIL")
PASS_WORD = os.getenv("PASSWORD")
FILE_DIR = os.getenv("FILE_DIR")
ANSWER_SHEET = os.getenv("ANSWERS")

### Commonly used informations
- `OPT`: Should be default "2" cuz I don't remember its use lol
- `EMAIL / PASSWORD`: The name is self-explanatory
- `FILE_DIR`: Should be written in Windows' format
- `MODE`: Should also default in "2" for linear scan mode
- `ANSWER_SHEET`: To use with MODE 2


In [ ]:
OPT = "2"
MODE = "2"


In [ ]:
currentDocxString = convertDoc2Txt(FILE_DIR)
currentDocxString = formatter(currentDocxString)
finalizedDictionary = filterer(ANSWER_SHEET, MODE, currentDocxString)

driver = webdriver.ChromiumEdge()
driver.get("https://www.quizizz.com/login")
actions = ActionChains(driver)

# start(driver, finalizedDictionary, mail, passwd)

### Signing in code block
- Extra Note: Skip the following block code if you want to sign-in using other options.
- `SIGNIN`: Basically the button that used to select the sign-in using Email option.
- `EMAIL_ENTRY`: The textbox that used to insert the user's email.
- `PASS_ENTRY`: The textbox that used to insert the user's password.

There're currently some problem with the password insertion progress

In [ ]:
SIGNIN = (
    r'//*[@id="base-0"]/div/div[2]/div[2]/section/section/section/section[1]/section/section[2]/section[1]/button[2]'
)

EMAIL_ENTRY = r'//*[@id="email-field-input"]'
EMAIL_CONT = r'//*[@id="base-0"]/div/div[2]/div[2]/section/section/section/section[1]/div/form/button[2]'
driver.find_element(By.XPATH, SIGNIN).click()
emailTxt = driver.find_element(By.XPATH, EMAIL_ENTRY)
emailTxt.clear()
emailTxt.send_keys(EMAIL)
driver.find_element(By.XPATH, EMAIL_CONT).click()

# PASS_ENTRY = r'//*[@id="password-field-input"]'
# passdTxt = driver.find_element(By.XPATH, PASS_ENTRY)
# passdTxt.clear()
# passdTxt.send_keys(PASS_WORD)
# driver.find_element(By.XPATH, EMAIL_CONT).click()
print(PASS_WORD)

In [ ]:
driver.get('https://quizizz.com/admin?modal=contentCreation&type=quiz')

NEW_QUIZ = r'//*[@id="app-body-container"]/section/div/div[2]/div/div[2]/div[1]/button'
WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.XPATH, NEW_QUIZ)))
# driver.find_element(By.XPATH, NEW_QUIZ).click()

In [ ]:
SUBMIT_BTN = r'//*[@id="question-editor-container-mobile"]/main/footer/button'
NEW_QUESTION = r'//*[@id="quiz-editor-content"]/section/section/div[4]/div/div/div/div[1]/div[1]/button'
ENTRY = r'//*[@id="quiz-editor-content"]/section/div[4]/div/div[2]/div/button'
NEW_LOOP = (
    r'//*[@id="modal-layer-2"]/div/div/div/body/div/div/div/div[2]/div[1]/button'
)
TITLE = (
    r'//*[@id="query-editor-tiptap-wrapper"]/div[2]'
)

In [ ]:



WebDriverWait(driver, 60).until(
    EC.presence_of_element_located(
        (By.XPATH, NEW_QUESTION))
)
driver.find_element(By.XPATH, NEW_QUESTION).click()

for questionStack in finalizedDictionary.values():
    # Interact
    driver.find_element(By.XPATH, TITLE).click()
    actions.send_keys(questionStack['title'])
    actions.perform()
    for i in range(len(questionStack['options'])):
        driver.find_element(By.XPATH, f'//*[@id="question-editor-container-mobile"]/main/div/section/div/div[2]/div[1]/div[{i+1}]/div[2]').click()
        actions.send_keys(questionStack['options'][i])
        actions.perform()
        if questionStack['options'][i] == questionStack['answer']:
            time.sleep(0.5)
            driver.find_element(By.XPATH, f'//*[@id="question-editor-container-mobile"]/main/div/section/div/div[2]/div[1]/div[{i+1}]/div[1]/button[2]').click()
            # time.sleep(0)
            # actions.key_down(Keys.CONTROL).send_keys("M").key_up(Keys.CONTROL).perform()
            # driver.execute_script('arguments[0].click();',
                                #   driver.find_element(By.XPATH, f'//*[@id="question-editor-container-mobile"]/main/div/section/div/div[2]/div[1]/div[{i+1}]/div[1]/button[3]'))
            # driver.find_element(By.XPATH,
            #                     f"/html/body/div[1]/div[1]/div/div[2]/main/div/section/div/div[2]/div[1]/div[{i+1}]/div[1]/button[3]").click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, SUBMIT_BTN).click()
    # actions.key_down(Keys.CONTROL).send_keys("S").key_up(Keys.CONTROL).perform()

    # ? ======================== ONE QUESTION LOOP DONE ===============

    # ? ======================== GENERATING A NEW CYCLE ===============
    if (questionStack['title'] ==
            finalizedDictionary[len(finalizedDictionary) - 1]['title']):
        break
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, ENTRY))
    )
    driver.execute_script('arguments[0].click();',
                            driver.find_element(By.XPATH, ENTRY))
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((
            By.XPATH, NEW_LOOP))
    )
    driver.execute_script(
        'arguments[0].click();',
        driver.find_element(By.XPATH, NEW_LOOP))